This tutorial presents how to analyze the results obtained from a Real Time Propagation calculation using the  $\delta$-kick technic to compute the frequency-dependent polarizability within the perturbative approximation, see https://www.cp2k.org/howto:delta_kick. 

This tutorial is structured as follows:

- Short theoretical reminders 
- Overview of the CP2K input file 
- How to open and perform a first analysis of the CP2K results to obtain the resonance frequency
- More details about how to extract the oscillator strength, here using a smoothing function
    
In this tutorial, we will study the response of a carbon-monoxide in the gas phase upon a $\delta$-kick along its perpendicular direction.
We will focus on the X-Ray range, but in principle, other ranges of frequency can be sampled using this technic. 


# Initialization

In [ ]:
# Loading some important packages and define some constants for unit conversions
%reset -f
import importlib
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

au_to_ev = 27.211324570273 # convert atomic units to electron volts (for frequencies)
au_to_Vm = 5.142206747E+11 # convert atomic units to V/m (for field)
au_to_D = (8.4783536255 / 3.335640952) # convert atomic units to Debye (for dipole)
au_to_fs = 0.02418884254 # convert atomic units to femtosecond (for time)
from_ev_to_nm_at_529 = 2.343746559546314 # used to convert frequency from ev to nm

L_xyz = ['x', 'y', 'z'] # will be used later on for plots


# Some theoretical reminders

**Fourier definitions** 

The Fourier transform of a function "f" from the time domain to the frequency one is:

$$
f(\omega) = \frac{1}{2 \pi} \int_{-\infty}^{+ \infty} f(t) e^{i \omega t} dt 
$$

And the other way from frequency to time:

$$
f(t) = \int_{-\infty}^{+ \infty} f(\omega) e^{-i \omega t} d\omega
$$


Note that the normalization factor comes from the fact that the $\delta$ dirac function is define using:

$$
2 \pi \delta(t-t_0) = \int_{-\infty}^{+ \infty} e^{-i \omega (t-t_0)} d\omega,  \ f(t) = \int_{-\infty}^{+ \infty} f(t') \delta(t-t') dt'
$$


**Response theory reminders**

In the perturbative regime and at the linear order, the time-dependent response of an electronic cloud can be decomposed in the Fourier space: the response at a given frequency is proportional to the perturbation acting on the system at the same frequency. In particular, the induced dipole moment (or equivalently the induced current) at a given frequency, $\mu^\omega$, is given by the product of the polarizability matrix, $\alpha(\omega, \omega)$, with the perturbative electric field at the same frequency, $F^\omega$, within the dipolar approximation.

$$
\mu^\omega = \alpha(\omega, \omega) \cdot F^\omega
$$

This quantity involves a dot product between the field vector and the polarizability matrix. 
Yet, in the following, we will drop the vectorial behaviour by assuming that the field is along one direction and that the polarizability matrix is diagonal

In Linear-Response-TDDFT approaches, one computes the excited state promoted by an electric field oscillating at a specific frequency $\omega$ and then computes the transition dipole moment associated with such electronic transition. This transition dipole moment is then used to compute the polarizability for resonant or non-resonant frequency. 


In the Real-Time approach presented here, we will excite **all** the possible electronic transitions at the beginning of the simulation and then propagate this excited electronic cloud to observe the induced dipole moment. We will deduce the polarizability tensor for any frequency from this induced dipole moment. 




**The** $\mathbf{\delta}$**-kick technic**

In the $\delta$-kick technic, the electronic structure is first obtained at the ground state, for instance using DFT, and then perturbed with an instantaneous electric field named a $\delta$-kick: 

$$
F(t) = F^0 \delta(t)
$$

The wave-function can be thus described to be the one at the ground state plus all the possible excited states since a time $\delta$-kick contains all the frequency: 

$$
F^\omega = \frac{F^0}{2 \pi} \int_{-\infty}^{+ \infty} \delta(t) e^{i \omega t} dt = \frac{F^0}{2 \pi} e^{i \omega \times 0} = \frac{F^0}{2 \pi}
$$

Therefore, the field amplitude in the Fourier space is $F^0 / 2 \pi$ for all frequencies.
This means that all the frequencies of the dipole moment are excited: 

$$
\mu^\omega = \frac{1}{2 \pi} \alpha(\omega, \omega)  F^0
$$

Hence, we will propagate this excited wave-function in time, compute the time-dependent dipole $\mu(t)$ (or current for condensed phase), and perform the Fourier transform to obtain $\mu^\omega$ to finally compute the polarizability $\alpha(\omega, \omega)$: 

$$
\text{Re} \left[ \alpha(\omega, \omega) \right]  = 2 \pi \frac{\text{Re} \left[ \mu^\omega \right] }{F^0} \\
\text{Im} \left[ \alpha(\omega, \omega) \right] = 2 \pi \frac{\text{Im}  \left[ \mu^\omega \right] }{F^0} 
$$

# CP2K input files

This has been done for isolated carbon-monoxide at the DFT/PBEh level using a PCSEG-2 basis set in CP2K.
One can look at the input file contains in RTP.inp, in the following we will emphasis a few important point related to the $\delta$-kick technic. 

**Time step** 

The time step used to propagate the wave-function is related to the frequency of interest. Typically, one should do a trade-off between the maximal frequency that can be sampled, this gives the time step value, and the accuracy of the sampling itself, related to the total simulation time. Typically, the time step should be about one order of magnitude lower then the frequency of interest. Here, we will tackle the Oxygen K-edge: the electronic transition between the Oxygen 1s and the first available excited state. 

According to LR-TDDFT calculation (using the XAS_TDP modulen see the wiki on cp2k.org), the first transition occurs at 529~ev with an oscillator strength of 0.044 a.u. A tutorial about XAS_TDP is available in the HOWTOs of the cp2k.org website: https://www.cp2k.org/howto:xas_tdp . Another tutorial about real-time TDDFT with an explicit field for carbone monoxide is also available on cp2k.org, https://www.cp2k.org/howto:rtp_field_xas, which provides more insight about such calculation and its connection with real-time method. 

In our $\delta$-kick approach, it means that the induced dipole moment should have an oscillatory component around the frequency corresponding to 529~ev. Hence, we should have a time step smaller then this frequency. Below how to find a good estimate of the time step for a given frequency in ev:


In [ ]:
# input:
omega_ev = 529 #the frequency of the targetted state in ev

# output
period_fs = 2*np.pi/(omega_ev/(au_to_ev*au_to_fs)) # the period in fs
delta_t = period_fs/10

print('A reasonable time step to sample a frequency of ' + str(omega_ev) + ' ev would be ' + str(delta_t)[:8] + ' femtosecond') 


Thus, a time step of 0.00078 femtosecond would lead to 10 time step per period for a oscillation at 529 ev. For this tutorial, we have use this value for the real time propagation calculation in CP2K: 

``
TIMESTEP [fs] 0.00078
``


**Field properties**


The field perturbation (the $\delta$-kick) is defined by its amplitude and polarization. 

The amplitude depends on the system of interest, typically $10^{-3}$ would do, and a good way to check it is to run 2 simulations: one with once the amplitude and another with twice the amplitude. Within the linear regime, the response of the system should have also doubled. 
If it is **not** the case, it means that the field is strong enough to triggers higher response order. Therefore, try to lower the field amplitude by one or two order of magnitude to recover the linear regime. 

Note that applying a too low field may lead to numerical noise. 
For isolated carbon monoxide, we have checked that $10^{-3}$ is within the linear regime. 

The polarization of the field determine which excited state is triggered: the electric field and the transition dipole moment should be non-perpendicular. One way to know how to select the polarization is to run a Linear-Response TDDFT calculation and to look at the transition dipole moment decomposition along the laboratory axis (or the oscillator strength decomposition). For instance, have a look at the first part of the tutorial on Real Time TDDFT with a time-dependent field for isolated carbone monoxide. Relying on this calculation, the electronic transition at 529 ev is perpendicular to the CO bond. For the geometry we are using, it means that the field polarization should be along x (or y). 

The corresponding input for CP2K are: 

``
DELTA_PULSE_DIRECTION 1 0 0
DELTA_PULSE_SCALE 0.001
``

**Note about the choice of the Gauge**

For isolated system, use the length implementation because the perturbation is applied up to all perturbative order: 

``
PERIODIC .FALSE.
``

For condensed phase system, use the velocity gauge instead

``
PERIODIC .TRUE.
``

In this case, the perturbation will be applied only within the first order. We have used the length one for this tutorial. 

# Read the RTP results and perform straightforward analysis

Now, let us read the results obtained from the Real Time Propagation starting from a $\delta$-kick perturbed wave-function.

To do so, we will rely on the python package cp2kRTPtools, https://github.com/glb96/cp2kRTPtools, developed by the CP2K community to help the reading of output file on python and to provide a few analyses, in particular Fourier transform. 
Of course, you can use your functions or preferred packages to perform the following analysis.

First, let us load the *cp2ktools* package  

In [ ]:
import cp2kRTPtools
print(importlib.reload(cp2kRTPtools))

from cp2kRTPtools import open_output_files
from cp2kRTPtools import ft_utils


## Read the results

The data for a $\delta$-kick along the $x$-direction should be available in the Datas directory in this folder.  

In [ ]:
# Set the path to the output files, should be in ./Datas
dirname =  './Data'

# DON'T MODIFY IF YOU ARE USING THE TUTORIAL OUTPUT
delta_t  = 0.00078/au_to_fs # in atomic unit
# Note that the "DELTA_PULSE_SCALE" is NOT the actual field amplitude applied. Have a look in the output file of CP2K to found the amplitude applied (just before the first time step)
F_0 = 0.33249185E-3    


The following lines read the evolution of the total electric energy and the dipole moment with respect to time


In [ ]:
# energy in atomic units
filename = dirname  + '/RTP.out'
print('Filename for the energy:', filename)
L_energy = open_output_files.read_energy(filename)[:]

# dipole moment, the cp2k default unit is Debye: converted into atomic unit
filename = dirname + '/dipole'
print('Filename for the dipole moment:', filename)
L_dipole_xyz = open_output_files.read_dipole_moment(filename)[1:]/au_to_D

# time list in atomic units
L_time = np.array([k*delta_t for k in range(1, 1+len(L_energy), 1)])


The following first plot shows the evolution of the total electronic energy during the simulation. One can check that the evolution of the total energy throughout the real-time propgation is very limited. This should be expected as the RTP scheme should conserve the energy. 

The second plot show the time-dependent dipole moment. There is almost no fluctuation along the $y$ and $z$ axis while the dipole along the $x$ axis fluctuates. It means that the polarizability is indeed diagonal for the perturbation along the $x$ direction. 

In [ ]:
L_delta_energy = L_energy - L_energy[0]
plt.figure(1, figsize=(10, 7))
plt.plot(L_time, L_delta_energy, lw=3)
plt.xlabel(r'Time [a.u.]' , fontsize=25)
plt.ylabel(r'$\Delta E$ [a.u.]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


plt.figure(2, figsize=(10, 7))
for axis in  range(3):
    plt.plot(L_time, L_dipole_xyz[:, axis], lw=3, label=L_xyz[axis])
plt.xlabel(r'Time [a.u.]' , fontsize=25)
plt.ylabel(r'$\mu(t)$ [a.u.]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


## Fourier Transform

From the time dependent dipole moment, we can now Fourier transform it to obtain the polarizability in the frequency range around 530 ev and check that this technics also show a resonances, as the Linear Response TDDFT one.

### Example

To perform the Fourier Transform, one can use the module ft_utils from cp2ktools. 

The Fourier transform is performed using the cos and sin function, not the FFT algorithm. 
Note that the Fourier transform is performed for the authorized frequency only:

$$
\omega T = n 2 \pi 
$$

Where $T$ is the total simulation time and $n$ is an integer. 
Hence, when one requires a range of frequencies, see below, the list of frequencies studies will be automatically determined. 

Please note that the ft_utils module has a normalization factor compared to the one proposed above: 

$$
C = \frac{2 \pi}{T}
$$

Therefore, the results obtained with this function would be normalized by the inverse of this factor. 

Note also that the frequencies are defined and plotted in electron volts, but all the calculations are done in atomic units. 

The following cell performs the Fourier transform of a cos function using the ft_utils module

In [ ]:
# Example: 
omega_try = 2*np.pi #the frequency of the cosinus
nbr_period = 18 # how many period is plotted in the time domain

# Time domain

L_t = np.linspace(0, nbr_period, 1000)
L_f = np.cos(omega_try*L_t) # the time dependent function to Fourier transform

plt.figure(1, figsize=(10, 7))
plt.plot(L_t, L_f.real, lw=3)
plt.xlabel(r'Time' , fontsize=25)
plt.ylabel(r'f(t)', fontsize=25)
plt.xticks( fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

# Fourier Transform

omega_min = -10*2*np.pi # minimal value for the frequency
omega_max = 10*2*np.pi # maximal value for the frequency 
N_omega = 200 #nbr of frequencies, note that this nbr may be reduced if the total time of the simulation is too small
 
# See the cp2ktools for this parameters. In short: 1: fast, 2: fast and more point in the frequency domain, -1: slow and very large number of point in the frequency domain
omega_precision = 1

# What to Fourier transform:
L_to_ft = L_f 

# The damping function, see later. Here we use unity to not use any damping.
L_damp = np.zeros(len(L_t)) + 1

# How to call the function to perform the Fourier transform. L_omega is the frequency list and L_ft the Fourier transform values
L_omega, L_ft = ft_utils.perform_ft(L_t, L_to_ft, L_damp, omega_min, omega_max, N_omega, omega_precision=omega_precision)

# Normalization
C_norm = 2*np.pi/(L_t[-1]-L_t[0])
L_f_ft = np.zeros(len(L_ft), dtype=complex)
L_f_ft.real = L_ft[:,0].real/C_norm
L_f_ft.imag = L_ft[:,0].imag/C_norm

In [ ]:
# The Fourier transform is supposed to be 2 delta diract at the +-omega_try frequency 
# The amplitude should be: nbr_period/(4 np.pi)

plt.figure(1, figsize=(10, 7))
plt.plot(L_omega, L_f_ft.real, lw=3, label=r'Re $f^\omega$')
plt.plot([-omega_try, omega_try], [nbr_period/(4*np.pi), nbr_period/(4*np.pi)], 'r*', ms=10, label=r'Expected value')
plt.xlabel(r'$\omega$ ' , fontsize=25)
plt.ylabel(r'Re $f^\omega$', fontsize=25)
plt.legend(fontsize=20)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(2, figsize=(10, 7))
plt.plot(L_omega, L_f_ft.imag, lw=3, label=r'Im $f^\omega$')
plt.plot(L_omega, L_omega*0, '--r', lw=3, label=r'Expected value')
plt.xlabel(r'$\omega$ ' , fontsize=25)
plt.ylabel(r'Im  $f^\omega$ ', fontsize=25)
plt.legend(fontsize=20)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


### Fourier Transform the time-dependent dipole moment

Now that we checked that the Fourier Transform works for simple function, let us Fourier transform the simulation dipole moment along the x direction in the frequency range around 529 ev

In [ ]:
omega_min = 526.5/au_to_ev # in atomic unit
omega_max = 531.5/au_to_ev # in atomic unit
N_omega = 45 #nbr of frequencies, note that this nbr may be reduced if the total time of the simulation is too small
 
# see the cp2ktools for this parameters. In short: 1: fast, 2: fast and more point in the frequency domain, -1: slow and very large number of point in the frequency domain
omega_precision = 2

# The dipole moment along x:
L_to_ft = L_dipole_xyz[:,0]

# the damping function, see later
L_damp = np.zeros(len(L_time)) + 1
        
L_omega, L_ft = ft_utils.perform_ft(L_time, L_to_ft, L_damp, omega_min, omega_max, N_omega, omega_precision=omega_precision)
        
# Normalization
C_norm = 2*np.pi/(L_time[-1]-L_time[0])
L_dipole_ft = np.zeros(len(L_ft), dtype=complex)
L_dipole_ft.real = L_ft[:,0].real/C_norm
L_dipole_ft.imag = L_ft[:,0].imag/C_norm

L_omega_ev = L_omega*au_to_ev


The following plots show the Real and Imaginary part of the Fourier transformed dipole moment in the 526.5 - 531.5 ev frequency range.

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, '-k', lw=3)
plt.plot(L_omega_ev, L_dipole_ft.real, lw=3)
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Re $\mu^\omega$', fontsize=25)
plt.xticks([527, 528, 529, 530, 531], fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


plt.figure(2, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, '-k', lw=3)
plt.plot(L_omega_ev, L_dipole_ft.imag, lw=3)
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Im  $\mu^\omega$ ', fontsize=25)
plt.xticks([527, 528, 529, 530, 531], fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)



Using the formula presented previously, we can extract the real and imaginary part of the polarizability (note that the Fourier transform of the electric field is pure real):

$$
\text{Re} \left[ \alpha(\omega, \omega) \right]  = 2 \pi \frac{\text{Re} \left[ \mu^\omega \right] }{F^0} \\
\text{Im} \left[ \alpha(\omega, \omega) \right] = 2 \pi \frac{\text{Im}  \left[ \mu^\omega \right] }{F^0} 
$$

In [ ]:
F_omega = F_0/(2*np.pi)
L_polarizability = np.zeros(len(L_dipole_ft), dtype=complex)
L_polarizability.real = L_dipole_ft.real/F_omega
L_polarizability.imag = L_dipole_ft.imag/F_omega

plt.figure(1, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, '-k', lw=3)
plt.plot(L_omega_ev, L_polarizability.real, lw=3)
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Re $\alpha$', fontsize=25)
plt.xticks([527, 528, 529, 530, 531], fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


plt.figure(2, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, '-k', lw=3)
plt.plot(L_omega_ev, L_polarizability.imag, lw=3)
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Im  $\alpha$ ', fontsize=25)
plt.xticks([527, 528, 529, 530, 531], fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

### Comparison with theoretical results

Here, one can see that the polarizability has a real an imaginary part with a pole at about 529 ev. 
This is coherent with the fact that the Oxygen K-edge (1s -> Lumo) of carbone monoxide frequency is suposed to be at 529 ev according to Linear-Response TDDFT calculation. 

In this case, the polarizability close to the resonance is suposed to be:

$$
\alpha(\omega, \omega) = \frac{\mu_{\omega_{res}}^2}{\omega - \omega_{res}} + \frac{i}{2 \pi} \mu_{\omega_{res}}^2 \delta(\omega - \omega_{res})
$$

Where $\omega_{res}$ is 529 ev and $\mu_{\omega_{res}}$ is the transition dipole moment. 


In [ ]:
def alpha_theoric_complex(L_omega, omega_res, mu_2):
    '''
    Return as a complex list the complex polarizability within the frequency range given by L_omega. 
    omega_res is the resonant frequency and mu_2 the associated transition dipole moment. 
    
    Note that a list can be given for omega_res and mu_2. In this case, the polarizability will be the sum over the different available transitions.
    '''
    L_alpha = np.zeros(len(L_omega), dtype=complex)
    if isinstance(omega_res, list):
        for k in range(len(omega_res)):
            L_alpha.real += mu_2[k]/(omega_res[k]-L_omega) 
            index_resonance = np.argmin(np.abs(np.array(L_omega)-omega_res[k]))
            L_alpha.imag[index_resonance] += mu_2[k]/(2*np.pi)
    else:
        L_alpha.real = mu_2/(omega_res-L_omega) 
        index_resonance = np.argmin(np.abs(np.array(L_omega)-omega_res))
        L_alpha.imag[index_resonance] = + mu_2/(2*np.pi)
    return(L_alpha)

We reminds that the oscillator strenght $f^{osc}$ is linked to the transition dipole moment: 

$$
f^{osc} = \frac{2}{3} \omega_{res} \mu_{\omega_{res}}^2
$$

Note that the LR-TDDFT calculation is done for 1 spin. Here, the perturbation has been applied to both spin: it is as if the transition dipole moment has been double. Therefore, the transition dipole moment would: 

$$
\left( \mu_{\omega_{res}}^{RTP} \right)^2 = 2 \times \left( \mu_{\omega_{res}}^{LR}\right)^2 = \frac{3 f^{osc}}{\omega_{res}} 
$$


In [ ]:
omega_res = 529/au_to_ev # in a.u.
mu_rtp_2 = 3*0.044/omega_res # in a.u. 

L_alpha_theoric_complex = alpha_theoric_complex(L_omega, omega_res, mu_rtp_2)

plt.figure(1, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, '-k', lw=3)
plt.plot(L_omega_ev, L_alpha_theoric_complex.real, '-', lw=3)
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Re $\alpha_{th}$', fontsize=25)
plt.xticks([ 528, 529, 530], fontsize=25)
plt.yticks(fontsize=25)
plt.ylim([-7, 7])
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(2, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, '-k', lw=3)
plt.plot(L_omega_ev, L_alpha_theoric_complex.imag, '*', ms=10)
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Imag $\alpha_{th}$', fontsize=25)
plt.xticks([528, 529, 530], fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

Now let us compare the expected and computed polarizabilities:

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, '-k', lw=3)
plt.plot([529.09], [0], 'ko', ms=10)
plt.plot(L_omega_ev, L_polarizability.real, '-', lw=3, label='Computed')
plt.plot(L_omega_ev, L_alpha_theoric_complex.real, '-', lw=3, label='Expected')
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Re $\alpha$', fontsize=25)
plt.xticks([527, 528, 529, 530, 531], fontsize=25)
plt.yticks(fontsize=25)
plt.ylim([-2, 2])
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(2, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, '-k', lw=3)
plt.plot(L_omega_ev, L_polarizability.imag, '-', lw=3, label='Computed')
plt.plot(L_omega_ev, L_alpha_theoric_complex.imag, '-', ms=10, label='Expected')
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Imag $\alpha_{th}$', fontsize=25)
plt.xticks([527, 528, 529, 530, 531], fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

The real part looks similar for the computed and expected of the polarizability, but not the imaginary one. 
Looking at the real part, we can see that the sign change occurs sigtly after 529 ev for the computed one. Such missmatch of a few tens of ev should be expect and reflect the error due to the choice of the method for computing the resonant frequency. According to the $\delta$-kick technic, the Oxygen K-edge is at 529.09 ev approximatly instead of 529 ev. 

Let us shift the expected spectra by 0.09 ev

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, '-k', lw=3)
plt.plot([529.09], [0], 'ko', ms=10)
plt.plot(L_omega_ev, L_polarizability.real, '-', lw=3, label='Computed')
plt.plot(L_omega_ev+0.09, L_alpha_theoric_complex.real, '-', lw=3, label='Expected')
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Re $\alpha$', fontsize=25)
plt.xticks([527, 528, 529, 530, 531], fontsize=25)
plt.yticks(fontsize=25)
plt.ylim([-2, 2])
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(2, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, '-k', lw=3)
plt.plot(L_omega_ev, L_polarizability.imag, '-', lw=3, label='Computed')
plt.plot(L_omega_ev+0.09, L_alpha_theoric_complex.imag, '-', ms=10, label='Expected')
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Imag $\alpha_{th}$', fontsize=25)
plt.xticks([527, 528, 529, 530, 531], fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

Then, the obtained real polarizability and the expected one are close one to another. One can still observe a small missmatch of amplitudes and some small high frequency modulation. 

However, the computed and expected imaginary part are still very different.
This problem is due to the fact that we have performed the Real-Time calculation only during a finite amount of time: the Fourier transfrom is done over a function defined between time 0 and time T.
In other words, we have not performed the Fourier Transform of the induced dipole, $\mu^\omega$ but the Fourier Transform of the dipole multiplied by a Windows function 'W'. The Fourier transform of the time product of the dipole moment with the windows function is noted $\tilde{\mu}^\omega$: 

$$
\tilde{\mu}^\omega = \frac{1}{2 \pi} \int_{0}^{T} \mu(t) e^{i \omega t} dt = \frac{1}{2 \pi} \int_{- \infty}^{+ \infty} W(t) \mu(t) e^{i \omega t} dt
$$

Where the windows function can be written using two Heaviside functions: $\Theta(t) \Theta(T-t)$

One can show that the Fourier transform for a product is the convolution of the Fourier transform:

$$
\frac{1}{2 \pi} \int_{- \infty}^{+ \infty} W(t) \mu(t) e^{i \omega t} dt = \int_{- \infty}^{+ \infty} W^{\omega'} \mu^{\omega-\omega'} d\omega'
$$

In this case, the Fourier Transform of the windows function is cardinal sinus, which lead to these high frequency oscillation for the fourier transform of the dipole moment. 

In order to have a smoother computed polarizability, we will choose another windows function which has a nicer Fourier Transform: a Gaussian. The goal here is to be able to quantitatively compare the expected and computed Fourier transformed induced dipole moment to check the transition dipole moment value.

# Fourier Transform with a designed smoothing function

Let us multiply our time-dependent induced dipole moment with a gaussian function which is zero for t=0 and t=T. 

For instance, using a Gaussian function centered at t=T/2 and with small enough widith $\sigma$:

$$
G(t) = \exp \left[ -\frac{1}{2} \left( \frac{t-\frac{T}{2}}{\sigma} \right)^2 \right]
$$

In [ ]:
# The total simulation time: 
T0 = L_time[-1]-L_time[0] # in a.u
sigma = 0.125*T0 # in a.u. 

L_gaussian = np.exp(-1/2*((L_time-T0/2)/sigma)**2)

plt.figure(1, figsize=(10, 7))
plt.plot(L_time, L_gaussian, lw=3)
plt.xlabel(r'Time [a.u.]' , fontsize=25)
plt.ylabel(r'$G(t)$ [a.u.]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)




Now, we multiply this window function to the time dependent dipole moment along the x axis:

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(L_time, L_gaussian*L_dipole_xyz[:, 0], lw=3)
plt.xlabel(r'Time [a.u.]' , fontsize=25)
plt.ylabel(r'$G(t) \mu(t)$ [a.u.]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


If one performes the Fourier transform of $G(t) \mu(t)$ defined between $[0, T]$, is would gives: 

$$
\tilde{\mu}^\omega = \frac{1}{2 \pi} \int_{0}^{T} G(t) \mu(t) e^{i \omega t} dt = \frac{1}{2 \pi} \int_{- \infty}^{+ \infty} G(t) \mu(t) e^{i \omega t} dt = \int_{- \infty}^{+ \infty} G^{\omega'} \mu^{\omega-\omega'} d\omega'
$$

because $G(t<0)=G(T>0) = 0$. 

The advantage here compared to the previous case is that we have a gentle analytical expression for $G^{\omega}$ :

$$
G^{\omega} = \frac{1}{2 \pi} \int_{- \infty}^{+ \infty} \exp \left[ -\frac{1}{2} \left( \frac{t-\frac{T}{2}}{\sigma} \right)^2 \right] e^{i \omega t} dt = \frac{\sigma}{\sqrt{2 \pi}} \exp \left[ -\frac{\omega^2 \sigma^2}{2} \right] e^{i \omega \frac{T}{2}}
$$

so that we can compute the expected shape of the induced dipole moment with the analytical polarizability formula.

In the following cell is checked the formula for the windows Fourier transform

In [ ]:
# Perform the Fourier transform of the window function
omega_min = -1/au_to_ev
omega_max = 1/au_to_ev
N_omega = 20 #nbr of frequencies, note that this nbr may be reduced if the total time of the simulation is too small
 
# see the cp2ktools for this parameters. In short: 1: fast, 2: fast and more point in the frequency domain, -1: slow and very large number of point in the frequency domain
omega_precision = 1

# The dipole moment along x:
L_to_ft = L_gaussian

# the damping function, see later
L_damp = np.zeros(len(L_time)) + 1
        
L_omega_direct, L_ft = ft_utils.perform_ft(L_time, L_to_ft, L_damp, omega_min, omega_max, N_omega, omega_precision=omega_precision)
 
    
# Normalization
C_norm = 2*np.pi/(L_time[-1]-L_time[0])
L_gaussian_ft_direct = np.zeros(len(L_ft), dtype=complex)
L_gaussian_ft_direct.real = L_ft[:,0].real/C_norm
L_gaussian_ft_direct.imag = L_ft[:,0].imag/C_norm


In [ ]:
# Analytical formula
def tf_gaussian_complex(L_omega, sigma, T0):
    '''
    Fourier transform value of a Gaussian with widith sigma and centered at T0. L_omega is the frequency. 
    '''
    L_gaussian_tf = sigma/(np.sqrt(2*np.pi))*np.exp(-1/2*(sigma*L_omega)**2)*np.exp(+1j*L_omega*T0/2)
    return(L_gaussian_tf)

L_omega_gaussian = np.linspace(-1, 1, 1000)/au_to_ev # in atomic unit
L_gaussian_tf = tf_gaussian_complex(L_omega_gaussian, sigma, T0) # the blur due to gaussian smearing


In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(L_omega_direct*au_to_ev, L_gaussian_ft_direct.real, '*', lw=3, ms=10)
plt.plot(L_omega_gaussian*au_to_ev, L_gaussian_tf.real, '-', lw=3, label='Real')
plt.plot(L_omega_direct*au_to_ev, L_gaussian_ft_direct.imag, '*', lw=3, ms=10)
plt.plot(L_omega_gaussian*au_to_ev, L_gaussian_tf.imag, '-', lw=3, label='Imag')
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'$G^\omega$ [a.u.]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


# checks amplitude with Perseval Theorem:
integral_time = np.sum(L_gaussian*np.conjugate(L_gaussian))*(L_time[1]-L_time[0])
integral_frequecy = (2*np.pi)*np.sum(L_gaussian_tf*np.conjugate(L_gaussian_tf))*(L_omega_gaussian[1]-L_omega_gaussian[0])

print('Should be very small:', (integral_frequecy-integral_time)/integral_frequecy)



Assuming that the ''true'' induced dipole moment in the frequency domaine, $\mu^\omega$, is given using the polarizability, we can predict the one computed using this gaussian smoothing: 

$$
\tilde{\mu}^\omega  = \int_{- \infty}^{+ \infty} G^{\omega'} \mu^{\omega-\omega'} d\omega' =  \frac{F^0}{2 \pi} \int_{- \infty}^{+ \infty} G^{\omega'} \alpha(\omega-\omega', \omega-\omega') d\omega' 
$$
where 
$$
\alpha(\omega, \omega) = \frac{\mu_{\omega_{res}}^2}{\omega - \omega_{res}} + \frac{i}{2 \pi} \mu_{\omega_{res}}^2 \delta(\omega - \omega_{res}).
$$

This lead for the real and imaginary part:

$$
\text{Re} \tilde{\mu}^\omega =  \frac{F^0 \mu_{\omega_{res}}^2 }{2 \pi} \int_{- \infty}^{+ \infty}  \frac{\text{Re} G^{\omega'}}{\omega-\omega' - \omega_{res}}  d\omega'  - \frac{F^0 \mu_{\omega_{res}}^2}{2 \pi} \text{Im} G^{\omega - \omega_{res}} \\
\text{Im} \tilde{\mu}^\omega =  \frac{F^0 \mu_{\omega_{res}}^2 }{2 \pi} \int_{- \infty}^{+ \infty}  \frac{\text{Im} G^{\omega'}}{\omega-\omega' - \omega_{res}}  d\omega'  + \frac{F^0 \mu_{\omega_{res}}^2}{2 \pi} \text{Re} G^{\omega - \omega_{res}} 
$$

We will perform this integration numerically, but first let us see how look like the direct Fourier transform of the time dependent dipole moment multply by the window:

In [ ]:
omega_min = 528/au_to_ev # in atomic unit
omega_max = 530/au_to_ev # in atomic unit


N_omega = 200 #nbr of frequencies, note that this nbr may be reduced if the total time of the simulation is too small
 
# see the cp2ktools for this parameters. In short: 1: fast, 2: fast and more point in the frequency domain, -1: slow and very large number of point in the frequency domain
omega_precision = -1

# The dipole moment along x:
L_to_ft = L_dipole_xyz[:,0]

# the damping function: the gaussian 
L_damp =  L_gaussian

L_omega, L_ft = ft_utils.perform_ft(L_time, L_to_ft, L_damp, omega_min, omega_max, N_omega, omega_precision=omega_precision)
       
    
# Normalization
C_norm = 2*np.pi/(L_time[-1]-L_time[0])
L_tilde_dipole_ft = np.zeros(len(L_ft), dtype=complex)
L_tilde_dipole_ft.real = L_ft[:,0].real/C_norm
L_tilde_dipole_ft.imag = L_ft[:,0].imag/C_norm

L_omega_ev = L_omega*au_to_ev


In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, 'k-', lw=3)
plt.plot(L_omega_ev, L_tilde_dipole_ft.real, lw=3, label='Real')
plt.plot(L_omega_ev, L_tilde_dipole_ft.imag, lw=3, label='Imag')
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Computed $\tilde{\mu}^\omega$', fontsize=25)
plt.legend(fontsize=20)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.xlim([528.1, 529.9])
plt.xticks([528.5, 529, 529.5], fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

print('A good estimate of the resonant frequency is to look at the maximal value of the imaginary part:', L_omega_ev[np.argmax(np.abs(L_tilde_dipole_ft.imag))])



In [ ]:
# The range where the Fourier transform of the gaussian is non-zero but is zero at the lowest and highest frequency
L_omega_gaussian = np.linspace(-1, 1, 1000)/au_to_ev # in atomic unit

# The omega_res value is the position of the maximal value of the imaginary part of the induced dipole moment, see the upper plot.
omega_res = 529.08844/au_to_ev # in a.u.
mu_rtp_2 = 3*0.044/omega_res # in a.u. 

# The frequency range where the induced dipole moment should be computed with respect to the resonnant frequency:
omega_min = -2/au_to_ev # in atomic unit
omega_max = 2/au_to_ev # in atomic unit



In [ ]:
L_gaussian_tf = tf_gaussian_complex(L_omega_gaussian, sigma, T0) # the blur due to gaussian smearing


d_omega = L_omega_gaussian[1]-L_omega_gaussian[0] # in atomic unts
n_point = np.int((omega_max-omega_min + (L_omega_gaussian[-1]-L_omega_gaussian[0]))/d_omega)
if isinstance(omega_res, list):
    L_omega_polarizability = np.array([omega_res[0]+omega_min+L_omega_gaussian[0] + d_omega*k for k in range(n_point)]) # in atomic units
else:
    L_omega_polarizability = np.array([omega_res+omega_min+L_omega_gaussian[0] + d_omega*k for k in range(n_point)]) # in atomic units

L_alpha_theoric_complex = alpha_theoric_complex(L_omega_polarizability, omega_res, mu_rtp_2) # all unit in a.u. for the polarizability value

plt.figure(1, figsize=(10, 7))
plt.plot(L_omega_gaussian*au_to_ev, L_gaussian_tf.real, lw=3, label='Real')
plt.plot(L_omega_gaussian*au_to_ev, L_gaussian_tf.imag, lw=3, label='Imag')
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'$G^\omega$ [a.u.]', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(2, figsize=(10, 7))
plt.plot(L_omega_polarizability*au_to_ev, L_alpha_theoric_complex.real, '-', lw=3)
plt.plot(L_omega_polarizability*au_to_ev, L_omega_polarizability*0, 'k-', lw=3)
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Re $\alpha_{th}$', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.ylim([-20, 20])
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(3, figsize=(10, 7))
plt.plot(L_omega_polarizability*au_to_ev, L_alpha_theoric_complex.imag, '*', ms=10)
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Imag $\alpha_{th}$', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

In [ ]:
def convolute_g_f(L_f, L_g, d_omega):
    '''
    Compute the convolution of g with f: 
    \int_{- \infty}^{+ \infty} g^{\omega'} f^{\omega-\omega'} d\omega' 
    
    Here we assume that the g function has a shorter frequency range then the f one: f is ''blured'' by the g function. 
    '''
    N_g = len(L_g)
    N_to_compute = len(L_f)-N_g
    L_convolution = np.zeros(N_to_compute, dtype=complex)
    # the real part of f is a usual function 
    for j in range(N_to_compute):
        for i in range(N_g):
            L_convolution.real[j] = L_convolution.real[j] + L_f.real[N_g+j-i]*L_g.real[i]
            L_convolution.imag[j] = L_convolution.imag[j] + L_f.real[N_g+j-i]*L_g.imag[i]
    L_convolution = L_convolution*d_omega
    # the imaginary part of f is delta dirac: no integration weight 
    for j in range(N_to_compute):
        for i in range(N_g):
            pass
            L_convolution.real[j] = L_convolution.real[j] - L_f.imag[N_g+j-i]*L_g.imag[i]
            L_convolution.imag[j] = L_convolution.imag[j] + L_f.imag[N_g+j-i]*L_g.real[i]
    return(L_convolution)


L_dipole_conv_ft = F_0/(2*np.pi)*convolute_g_f(L_alpha_theoric_complex, L_gaussian_tf, d_omega)

if isinstance(omega_res, list):
    L_omega_conv = np.array([omega_res[0]+omega_min + d_omega*k for k in range(len(L_dipole_conv_ft))]) #in atomic units
else:
    L_omega_conv = np.array([omega_res+omega_min + d_omega*k for k in range(len(L_dipole_conv_ft))]) #in atomic units


In [ ]:

plt.figure(1, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, 'k-', lw=3)
plt.plot(L_omega_conv*au_to_ev, L_dipole_conv_ft.real, lw=3, label='Real')
plt.plot(L_omega_conv*au_to_ev, L_dipole_conv_ft.imag, lw=3, label='Imag')
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Predicted $\tilde{\mu}^\omega$ [a.u.]', fontsize=25)
plt.xlim([528.1, 529.9])
plt.xticks([528.5, 529, 529.5], fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


Finally we can compare the analytical formula and the one obtained by the $\delta$-kick approach:

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, 'k-', lw=3)
plt.plot(L_omega_ev, L_tilde_dipole_ft.real, lw=3, label='Computed')
plt.plot(L_omega_conv*au_to_ev, L_dipole_conv_ft.real, '--', lw=3, label='Predicted')
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Re $\tilde{\mu}^\omega$', fontsize=25)
plt.xlim([528.1, 529.9])
plt.xticks([528.5, 529, 529.5], fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


print('Ratio max predicted / max computed real part:', np.max(L_dipole_conv_ft.real)/np.max(L_tilde_dipole_ft.real))

plt.figure(2, figsize=(10, 7))
plt.plot(L_omega_ev, L_omega_ev*0, 'k-', lw=3)
plt.plot(L_omega_ev, L_tilde_dipole_ft.imag, lw=3, label='Computed')
plt.plot(L_omega_conv*au_to_ev, L_dipole_conv_ft.imag, '--', lw=3, label='Predicted')
plt.xlabel(r'$\omega$ [ev]' , fontsize=25)
plt.ylabel(r'Im  $\tilde{\mu}^\omega$ ', fontsize=25)
plt.xlim([528.1, 529.9])
plt.xticks([528.5, 529, 529.5], fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=20)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

print('Ratio max prediced / max computed imag part:', np.max(L_dipole_conv_ft.imag)/np.max(L_tilde_dipole_ft.imag))


This time, the comparison is excellent both for the resonant frequency position and the amplitude. 
Thus, the frequency is indeed about 529 ev and the oscillator strength 0.044 a.u. for the Oxygen K-edge of isolated carbon monoxide with these parameters. 

Note that this last part using a chosen windows for the Fourier transform is not required to compare the computed absorption spectra with experiments or another reference. This approach is one way to get an absolute value for the oscillator strength. Note also that such approach will be more difficult to apply if many excited state are close in energy.  